# OpenCV Face Extraction

This code can be used to extract faces from videos. The code works by getting individual frames one by one from the video and then extracts faces from the individual frames. This code extracts faces from every frame within the video, but can be changed to extract faces from different intervals. 

OpenCV's CNN based face detection model is used to detect faces as it is robust against large poses or faces at distance.

In [1]:
import dlib
import cv2
import os
import re
import json
import glob
import numpy as np
import random
from pylab import *
import pandas as pd
import tensorflow as tf
from PIL import Image, ImageChops, ImageEnhance

In [2]:
import logging
tf.get_logger().setLevel(logging.ERROR)

In [3]:
# get video folders in list
# expects each video in separate folder, with folder having the same name as the video

test_folders_list = sorted(glob.glob("videos/*/"))
(test_folders_list[0])

'videos/vid_1/'

In [4]:
# load opencv cnn face detector
net = cv2.dnn.readNetFromCaffe('OpenCV_CNN/deploy.prototxt.txt', 'OpenCV_CNN/res10_300x300_ssd_iter_140000.caffemodel')

# parent directory to store extracted face frames
parent_dir = "extracted_faces/"

for i in range(len(test_folders_list)):
    # create folders to save extracted face frames
    path = os.path.join(parent_dir, test_folders_list[i].split('/')[-2] + '.mp4').split('.')[0]
    if not os.path.exists(path):
        os.makedirs(path)
    count = 0
    # get video location to extract faces
    vid = (test_folders_list[i]) + test_folders_list[i].split('/')[-2] + '.mp4'
    video_capture = cv2.VideoCapture(vid)
    frameRate = video_capture.get(5)
    while video_capture.isOpened():
        frameId = video_capture.get(1)
        ret, frame = video_capture.read()
        if ret != True:
            break
        if frameId % ((int(frameRate+1))*1) != 0:
            (h, w) = frame.shape[:2]
            blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
            net.setInput(blob)
            detections = net.forward()
#             for i, d in enumerate(face_rects):
            for i in range(0, detections.shape[2]):
                confidence = detections[0, 0, i, 2]
                if confidence > 0.5:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")
                    if startX is None or startY is None or endX is None or endY is None:
                        break
                    else:
                        try:
                            # extract first 10 frames from the video 
                            if count<10:
                                # save frames, image size = 299x299
                                cv2.imwrite(str(path)+'/' + vid.split('.')[0].split('/')[-1] + "_" + str(count)+".png", cv2.resize(frame[startY-15:endY+15, startX-15:endX+15], (299,299), cv2.INTER_AREA))
                                count+=1
                            else:
                                video_capture.release()
                                break
                        except Exception as e:
                            break
    video_capture.release()